In [1]:
import os
import datetime as dt
import pandas as pd
import csv

In [2]:
# add the 'src' directory as one where we can import modules
import sys
import pathlib # __file__ isn't recognized in Jupyter, so we need this to get the root path
path = pathlib.Path.cwd()
PROJ_ROOT = path.parent 
src_dir = str(PROJ_ROOT / 'src')
PROJ_ROOT = str(PROJ_ROOT)
sys.path.append(src_dir)
# from data import make_dataset

In [17]:
from data import user_df_setup

interim_data_file_path = os.path.join(PROJ_ROOT,
                                 "data",
                                 "interim",
                                 "full_users_df.pkl")
users_df = pd.read_pickle(interim_data_file_path)
raw_data_file_path = os.path.join(PROJ_ROOT,
                             "data",
                             "raw",
                             "users_df.pkl")
raw_users_df = pd.read_pickle(raw_data_file_path)

all_users_df = user_df_setup.user_df_setup(raw_data_file_path, interim_data_file_path)

users_df = all_users_df
# raw_users_df
# raw_users_df[['username', 'timeCreated']].sort_values('timeCreated', ascending = False).head(20)
all_users_df.sort_values('date_created', ascending = False).head(20)
# all_users_df.index
# all_users_df.head(20)

,date_created,refresh_time,userId,unrated_threshold,risky_threshold,supportive_threshold
username,,,,,,
nimaghafari,2018-11-20,NaN,5bf47f7df434f50010c2550e,0.0,1.0,3.0
google,2018-11-20,NaN,5bf47741f434f50010c25507,0.0,1.0,3.0
shanila,2018-11-20,NaN,5bf46c4ff434f50010c25502,0.0,1.0,3.0
jmeow,2018-11-20,NaN,5bf3d03b0760320010e58efe,0.0,1.0,3.0
plush.mary,2018-11-20,NaN,5bf3a4fe0760320010e58ee3,0.0,1.0,3.0
nepthys,2018-11-20,NaN,5bf371bbf434f50010c254d0,0.0,1.0,3.0
philoiz,2018-11-19,NaN,5bf30d1a0760320010e58ec0,0.0,1.0,3.0
upstate518,2018-11-19,NaN,5bf2de44f434f50010c254a3,0.0,1.0,3.0
fenderman123,2018-11-19,NaN,5bf3028cf434f50010c254ad,0.0,1.0,3.0


In [4]:
current_users = all_users_df[all_users_df['date_created'] > (dt.date.today() - dt.timedelta(30))]
user_ids = all_users_df['userId'].tolist()
user_ids

['000000000000000000000000',
 '5aebcb8cc0162e0010966860',
 '5b0d91a3876ddd001af27ccb',
 '5b198f09dfd99a00113eefcd',
 '5b23b05adfd99a00113eefd9',
 '5b26d5de04dd5a001abdcf5a',
 '5b26dba804dd5a001abdcf5d',
 '5b26dd7804dd5a001abdcf62',
 '5b29855504dd5a001abdcf84',
 '5b2b007a04dd5a001abdcf9e',
 '5b2b9cb204dd5a001abdcfa4',
 '5b2b9efc04dd5a001abdcfaa',
 '5b2bd36804dd5a001abdcfc8',
 '5b30f032a58439001a5e49a3',
 '5b33ab0da58439001a5e49e9',
 '5b34d20aa58439001a5e49ef',
 '5b354219a58439001a5e49fb',
 '5b354dada58439001a5e49fe',
 '5b35544fa58439001a5e4a08',
 '5b3682b7a58439001a5e4ab0',
 '5b630a9cb7c059001a4dbad4',
 '5b647ca6d684d6001a2249a6',
 '5b6ef82c78c5fb001a2216f1',
 '5b6f42096d0afc001a8a8f71',
 '5b72d27e6ae794001ab18e56',
 '5b85af6ebef5f7001a6464fc',
 '5b91efc055007d0010817fa0',
 '5b95757fad7027001080b22c',
 '5b97e311ad7027001080b26d',
 '5b97e336ad7027001080b270',
 '5b988229ad7027001080b2b8',
 '5b9f052e3ed631001a47c2de',
 '5ba05e113ed631001a47c354',
 '5ba05fc23ed631001a47c35d',
 '5ba061173ed6

### Force-pulling Contacts data from MongoDB

In [5]:
from data import database_query

raw_data_path = os.path.join(PROJ_ROOT,
                             "data",
                             "raw")
current_users_df = all_users_df[all_users_df['date_created'] > (dt.date.today() - dt.timedelta(30))]
user_ids = current_users_df['userId'].tolist()
database_query.make_raw_contacts_df(database_query.mongo_connect(),
                                   raw_data_path,
                                   user_ids)

### Force-creating Contacts interim data

In [6]:
from data import contacts_df_setup

interim_data_path = os.path.join(PROJ_ROOT,
                                 "data",
                                 "interim")
contacts_dict = {}

usernames = current_users.index
for username in usernames: 
    interim_contact_data_file_path = os.path.join(interim_data_path, 'contacts_df_' + username + '.pkl')
    if os.path.isfile(interim_contact_data_file_path):
        contacts_df = pd.read_pickle(interim_contact_data_file_path)
    else:
        contacts_df = contacts_df_setup.contacts_df_setup(username,
                                                          current_users_df,
                                                          raw_data_path,
                                                          interim_data_path)
    contacts_dict[username] = contacts_df

In [7]:
import matplotlib.pyplot as plt

f, axarr = plt.subplots(len(usernames),figsize=(10,30), sharex = True, sharey = True)
plt.tight_layout()

for count, e in enumerate(usernames):  
    unrated_threshold = users_df.loc[e, 'unrated_threshold']
    risky_threshold = users_df.loc[e, 'risky_threshold']
    supportive_threshold = users_df.loc[e, 'supportive_threshold']
    
    contacts_df = contacts_dict[e].sort_values('score', ascending = False)
    unrated = contacts_df['score'][contacts_df['score'] < unrated_threshold]
    risky = contacts_df['score'][(contacts_df['score'] >= unrated_threshold)
                                                     & (contacts_df['score'] <= risky_threshold)]
    neutral = contacts_df['score'][(contacts_df['score'] > risky_threshold)
                                                     & (contacts_df['score'] < supportive_threshold)]
    supportive = contacts_df['score'][(contacts_df['score'] >= supportive_threshold)]
    
    axarr[count].bar(supportive.index, supportive, 0.7, color = '#00cc00')
    axarr[count].bar(neutral.index, neutral, 0.7, color = 'b')
    axarr[count].bar(risky.index, risky, 0.7, color = '#ff6600')
    axarr[count].bar(unrated.index, unrated, 0.7, color = '#C0C0C0')
    axarr[count].set_title(e)
    
    # unrelated, pulling the counts
    current_users.loc[e, 'unrated_contacts'] = len(unrated)
    current_users.loc[e, 'risky_contacts'] = len(risky)
    current_users.loc[e, 'neutral_contacts'] = len(neutral)
    current_users.loc[e, 'supportive_contacts'] = len(supportive)
    current_users.loc[e, 'rated_contacts'] = len(contacts_df) - current_users.loc[e, 'unrated_contacts']
plt.show()

C:\Python35-32\lib\site-packages\pandas\core\indexing.py:357: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
C:\Python35-32\lib\site-packages\pandas\core\indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


<Figure size 1000x3000 with 36 Axes>

In [8]:
# current_users[['date_created', 'unrated_contacts', 'risky_contacts', 'neutral_contacts', 'supportive_contacts', 'rated_contacts']]
# current_users

### Force-pulling Locations data from MongoDB

In [9]:
database_query.make_raw_location_df(database_query.mongo_connect(),
                                   raw_data_path,
                                   user_ids)

### Force-creating Locations interim data

In [10]:
from data import locations_df_setup

locations_dict = {}

for username in usernames: 
    interim_loc_data_file_path = os.path.join(interim_data_path, 'locations_df_' + username + '.pkl')
    locations_df = locations_df_setup.locations_df_setup(username,
                                                          current_users_df,
                                                          raw_data_path,
                                                          interim_data_path)
    locations_dict[username] = locations_df
    current_users.loc[username, 'locations'] = len(locations_df.index)
    current_users.loc[username, 'safe_locations'] = len(locations_df[locations_df['type'] == 'safe'])
    current_users.loc[username, 'risky_locations'] = len(locations_df[locations_df['type'] == 'risky'])

C:\Python35-32\lib\site-packages\pandas\core\indexing.py:357: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
C:\Python35-32\lib\site-packages\pandas\core\indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [11]:
locations_df = locations_dict['+vinny']
len(locations_df)

15

In [12]:
locations_dict

{'+vinny':                                                   _id   type
 _id                                                         
 5bec3c5b0eb5fd0010495e3d  b'5bec3c5b0eb5fd0010495e3d'   safe
 5bec3c5c1dc2970010696df5  b'5bec3c5c1dc2970010696df5'   safe
 5bec3c5d1dc2970010696df6  b'5bec3c5d1dc2970010696df6'   safe
 5bec3c5e1dc2970010696df7  b'5bec3c5e1dc2970010696df7'   safe
 5bec3c5e0eb5fd0010495e3e  b'5bec3c5e0eb5fd0010495e3e'   safe
 5bec3c5f1dc2970010696df8  b'5bec3c5f1dc2970010696df8'   safe
 5bec3c5f1dc2970010696df9  b'5bec3c5f1dc2970010696df9'   safe
 5bec3c600eb5fd0010495e3f  b'5bec3c600eb5fd0010495e3f'   safe
 5bec3c610eb5fd0010495e40  b'5bec3c610eb5fd0010495e40'   safe
 5bec3c611dc2970010696dfa  b'5bec3c611dc2970010696dfa'   safe
 5bec3c621dc2970010696dfb  b'5bec3c621dc2970010696dfb'  risky
 5bec3c880eb5fd0010495e43  b'5bec3c880eb5fd0010495e43'  risky
 5bec3c930eb5fd0010495e44  b'5bec3c930eb5fd0010495e44'  risky
 5bec3c930eb5fd0010495e45  b'5bec3c930eb5fd0010495e45'  risk

In [13]:
current_users[['safe_locations','risky_locations']]

,safe_locations,risky_locations
username,,
vinoct24,10.0,1.0
test22,17.0,0.0
test26,1.0,0.0
vinoct31,10.0,1.0
emily3,2.0,0.0
giftest,0.0,0.0
giftest2,0.0,0.0
test29,0.0,0.0
chshanahan,19.0,0.0


In [14]:
from data import make_dataset

notification_dict = make_dataset.make_notification_data(users_df, usernames, PROJ_ROOT)
notification_dict.keys()

Username not found: []
5bd0a014dc3f590016b44f5a
                             timestamp                        type  \
_id                                                                  
5bd0bfb5738012001662351e  1.540407e+12            sent-sms-warning   
5bd0bfb6dc3f590016b44f6e  1.540407e+12  notification-task-question   
5bd0bfc0dc3f590016b44f6f  1.540407e+12  notification-task-question   
5bd0d745dc3f590016b44f78  1.540413e+12  notification-task-question   
5bd0e991738012001662352a  1.540418e+12         daily-task-question   

                                            userId  
_id                                                 
5bd0bfb5738012001662351e  5bd0a014dc3f590016b44f5a  
5bd0bfb6dc3f590016b44f6e  5bd0a014dc3f590016b44f5a  
5bd0bfc0dc3f590016b44f6f  5bd0a014dc3f590016b44f5a  
5bd0d745dc3f590016b44f78  5bd0a014dc3f590016b44f5a  
5bd0e991738012001662352a  5bd0a014dc3f590016b44f5a  
5bd7ac1b8765970016f4e824
                             timestamp                           

                             timestamp                 type  \
_id                                                           
5beb496f0eb5fd0010495e30  1.542146e+12  daily-task-question   
5bedec7c1dc2970010696e42  1.542319e+12  daily-task-question   
5bf08f5f0eb5fd0010495ec0  1.542492e+12  daily-task-question   
5bf332530760320010e58ecf  1.542665e+12  daily-task-question   
5bf483d1f434f50010c25512  1.542751e+12  first-week-question   

                                            userId  
_id                                                 
5beb496f0eb5fd0010495e30  5beb26500eb5fd0010495e08  
5bedec7c1dc2970010696e42  5beb26500eb5fd0010495e08  
5bf08f5f0eb5fd0010495ec0  5beb26500eb5fd0010495e08  
5bf332530760320010e58ecf  5beb26500eb5fd0010495e08  
5bf483d1f434f50010c25512  5beb26500eb5fd0010495e08  
5bec3a6b1dc2970010696def
                             timestamp                 type  \
_id                                                           
5bec3d821dc2970010696e0c  1.542209e+

dict_keys(['testtest', 'upstate518', 'jake', 'nepthys', 'joeybars7', 'hayyogirl', 'shanila', 'jmeow', 'rusty', 'vinnov10', 'test22', 'anonysauce', 'vinoct31', 'emily3', 'keirmaley', 'vinoct24', 'plush.mary', 'test26', 'giftest2', 'chshanahan', 'username', 'rabbit', '+vinny'])

In [15]:
import numpy as np

for username in usernames: 
    if username in notification_dict.keys():
        notifications_df = notification_dict[username]
        current_users.loc[username, 'notifications'] = len(notifications_df.index)
    else:
        current_users.loc[username, 'notifications'] = 0

C:\Python35-32\lib\site-packages\pandas\core\indexing.py:357: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
C:\Python35-32\lib\site-packages\pandas\core\indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [16]:
current_users['notifications']

username
vinoct24         23.0
test22            8.0
test26           49.0
vinoct31          7.0
emily3          181.0
giftest           0.0
giftest2         18.0
test29            0.0
chshanahan       23.0
vinnov8           0.0
vinnov10         12.0
altaomet          0.0
sirex             0.0
c13               0.0
testtest          3.0
inreverie         0.0
jake             23.0
username         15.0
rabbit            3.0
hayyogirl         7.0
+vinny           10.0
joeybars7         2.0
joeybars          0.0
rusty             1.0
upstate518        2.0
fenderman123      0.0
philoiz           0.0
ceecee            0.0
anonysauce        2.0
keirmaley         1.0
nepthys           3.0
plush.mary        9.0
jmeow             1.0
shanila           3.0
google            0.0
nimaghafari       0.0
Name: notifications, dtype: float64